In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interact, IntSlider, FloatSlider, fixed 
from IPython.display import display # Nécessaire pour afficher explicitement dans certains environnements

# Note: Ce code est conçu pour être exécuté dans un environnement Jupyter Notebook/Lab

def construire_hamiltonien_zigzag(k, N, t=1.0):
    """
    Construit la matrice Hamiltonienne H(k) pour un nanoruban zigzag.
    (Identique à la version précédente)
    """
    dim = 2 * N
    H = np.zeros((dim, dim), dtype=complex)
    gamma_k = 1 + np.exp(-1j * k)
    gamma_k_conj = np.conjugate(gamma_k)

    for m in range(N):
        idx_a = 2 * m
        idx_b = 2 * m + 1
        H[idx_a, idx_b] = -t * gamma_k
        H[idx_b, idx_a] = -t * gamma_k_conj

    for m in range(N - 1):
        idx_b_m = 2 * m + 1
        idx_a_m_plus_1 = 2 * (m + 1)
        H[idx_a_m_plus_1, idx_b_m] = -t
        H[idx_b_m, idx_a_m_plus_1] = -t

    return H

def calculer_bandes_zigzag(N, t=1.0, num_k_points=1000):
    """
    Calcule les énergies (valeurs propres) pour une plage de k de -pi à pi.

    Args:
        N (int): Largeur du ruban.
        t (float): Intégrale de saut.
        num_k_points (int): Nombre de points k à calculer entre -pi et pi.

    Returns:
        tuple: (k_values, eigenvalues)
            k_values (numpy.ndarray): Les valeurs de k utilisées.
            eigenvalues (numpy.ndarray): Tableau (num_k_points x 2N) des énergies.
    """
    # Modification: k va de -pi à pi
    k_values = np.linspace(-np.pi, np.pi, num_k_points)
    eigenvalues = np.zeros((num_k_points, 2 * N))

    for i, k in enumerate(k_values):
        Hk = construire_hamiltonien_zigzag(k, N, t)
        vals = np.linalg.eigvalsh(Hk)
        eigenvalues[i, :] = vals

    return k_values, eigenvalues

def tracer_bandes(k_values, eigenvalues, N, t=1.0):
    """
    Trace la structure de bandes de -pi à pi.

    Args:
        k_values (numpy.ndarray): Valeurs de k.
        eigenvalues (numpy.ndarray): Énergies correspondantes.
        N (int): Largeur du ruban (pour le titre).
        t (float): Valeur de t (pour l'échelle de l'axe y).
    """
    num_bands = eigenvalues.shape[1]
    # Afficher k en unités de pi/a (ici a=1)
    k_labels = k_values / np.pi

    plt.figure(figsize=(8, 6)) # Légèrement plus large pour la plage étendue
    for i in range(num_bands):
        plt.plot(k_labels, eigenvalues[:, i] / t, color='blue', linewidth=1.5)

    plt.xlabel(r'$k a / \pi$') # a=1 implicitement
    plt.ylabel(r'$E / t$')
    plt.title(f'Structure de Bandes - Nanoruban Zigzag (N = {N})')
    # Modification: xlim va de -1 à 1
    plt.xlim(-1, 1)
    min_E = np.min(eigenvalues / t)
    max_E = np.max(eigenvalues / t)
    plt.ylim(min(min_E - 0.2, -t*3.2), max(max_E + 0.2, t*3.2))
    plt.axhline(0, color='red', linestyle='--', linewidth=1, label='E = 0')
    # Ajouter des lignes verticales pour les points de haute symétrie 
    plt.axvline(x=-2/3, color='gray', linestyle=':', linewidth=0.8)
    plt.axvline(x=2/3, color='gray', linestyle=':', linewidth=0.8)
    plt.xticks([-1, -2/3, 0, 2/3, 1], [r'$-\pi$', r'$-2\pi/3$', '0', r'$2\pi/3$', r'$\pi$']) # Ticks personnalisés
    plt.grid(True, linestyle=':', alpha=0.7)
    plt.legend()
    plt.show() # Important pour afficher le graphique dans le contexte interactif

# --- Fonction pour l'interaction ---
def plot_interactive_bands(N, t=1.0, num_k_points=1000):
    """
    Fonction appelée par interact pour calculer et tracer les bandes pour une largeur N donnée.
    """
    k_vec, E_nk = calculer_bandes_zigzag(N, t=t, num_k_points=num_k_points)
    tracer_bandes(k_vec, E_nk, N, t=t)

# --- Création du Widget Interactif ---

# Définir le slider pour N
n_slider = IntSlider(
    min=2,         # Largeur minimale (au moins 2 pour avoir des bords)
    max=75,        # Largeur maximale
    step=1,        # Incrément
    value=10,      # Valeur initiale
    description='Largeur N:',
    continuous_update=False # Mettre à jour uniquement lorsque le slider est relâché
)

# Utiliser interact pour lier le slider à la fonction de tracé
interactive_plot = interact(
    plot_interactive_bands,
    N=n_slider,
    t=fixed(1.0), 
    num_k_points=fixed(1000) # Garder le nombre de points k fixe
)

interactive(children=(IntSlider(value=10, continuous_update=False, description='Largeur N:', max=50, min=2), O…